In [1]:
import psycopg2
import pandas as pd
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import plotly.graph_objs as go
import dash_bootstrap_components as dbc

In [2]:
#Database Connection

class DataBaseConnection:

    def __init__(self, database, host, port, user, password):

        self.database = database
        self.host = host
        self.port = port
        self.user = user
        self.password = password

        try:
            self.connection = psycopg2.connect(host=self.host, port = self.port, database=self.database, user = self.user, password = self.password)
            self.connection.autocommit = True
            self.cursor = self.connection.cursor()

        except ValueError:
            print('Can not connect to Database')

    def create_table(self, sql_query):
        self.sql_query = sql_query
        self.cursor.execute(sql_query)

    def alter_table(self, sql_query):
        self.sql_query = sql_query
        self.cursor.execute(sql_query)

    def import_from_CSV(self, sql_query):
        self.sql_query = sql_query
        self.cursor.execute(sql_query)

    def read_database(self, sql_query):
        self.sql_query = sql_query
        table = pd.read_sql_query(sql_query, self.connection)
        return table

    def insert_database(self, sql_query):
        self.sql_query = sql_query
        self.cursor.execute(sql_query)

In [3]:
data_base = DataBaseConnection(database = 'axel', 
                        host = 'localhost', 
                        port = '5432', 
                        user = 'postgres', 
                        password = 'Paranthaman12!'
                    )

In [4]:
#SQL_QUERY

sql_query = "SELECT "\
                "sp_perf_1.year, "\
                "sp_perf_1.month, "\
                "sp_perf_1.week, "\
                "(sp_perf_0.strategy_long_only_1 + sp_perf_1.strategy_long_only_1 + sp_perf_0.strategy_long_only_2 + " \
                "sp_perf_1.strategy_long_only_2)/4 as avg_strategy_long, " \
                "(sp_perf_0.strategy_short_only_1 + sp_perf_1.strategy_short_only_1 + sp_perf_0.strategy_short_only_2 + " \
                "sp_perf_1.strategy_short_only_2)/4 as avg_strategy_short, "\
                "((sp_perf_0.strategy_long_only_1 + sp_perf_1.strategy_long_only_1 + "\
                "sp_perf_0.strategy_long_only_2 + sp_perf_1.strategy_long_only_2)/4) + "\
                "((sp_perf_0.strategy_short_only_1 + sp_perf_1.strategy_short_only_1 + "\
                "sp_perf_0.strategy_short_only_2 + sp_perf_1.strategy_short_only_2)/4) as both_strategy "\
            "FROM "\
                "sp_perf_0, "\
                "sp_perf_1 "\
            "WHERE "\
                "sp_perf_0.year = sp_perf_1.year "\
                "and sp_perf_0.month = sp_perf_1.month "\
                "and sp_perf_0.week = sp_perf_1.week "\
            "ORDER BY "\
                "sp_perf_0.year, "\
                "sp_perf_1.year, "\
                "CASE "\
                    "when sp_perf_1.month = 'january' then 1 "\
                    "when sp_perf_1.month = 'february' then 2 "\
                    "when sp_perf_1.month = 'march' then 3 "\
                    "when sp_perf_1.month = 'april' then 4 "\
                    "when sp_perf_1.month = 'may' then 5 "\
                    "when sp_perf_1.month = 'june' then 6 "\
                    "when sp_perf_1.month = 'july' then 7 "\
                    "when sp_perf_1.month = 'august' then 8 "\
                    "when sp_perf_1.month = 'september' then 9 "\
                    "when sp_perf_1.month = 'october' then 10 "\
                    "when sp_perf_1.month = 'november' then 11 "\
                    "else 12 "\
                "End, "\
                "sp_perf_0.week, "\
                "sp_perf_1.week;"

In [5]:
#Data recovery

Data = data_base.read_database(sql_query = sql_query)

/var/folders/ql/mqkl0r054w78dm4sk7mqr8gc0000gn/T/ipykernel_48846/2136578319.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table = pd.read_sql_query(sql_query, self.connection)


In [6]:
#list of years to creat the slider

YEARS = Data['year'].unique()
number_of_years = len(YEARS)

# Dash and Plotly

In [ ]:
#Creat Dash application
app = Dash(__name__)

#Defind the app's layout
app.layout = html.Div(
    style = {'display':'flex',
             'justifyContent': 'space-between',
             'margin': '30px'
            },
            children = [
                html.Div(
                    style={'flex': '40%'},
                    children = [
                        html.H3(
                            'Historical performance',
                            style={'textAlign': 'left',
                                   'color': '#345beb',
                                   'font-weight': 900,
                                   'textDecoration': 'underline',
                                   'marginBottom': '20px',
                                   'text-decoration-color': '#edeae1'
                            }
                        ),
                        html.Br(0.7),
                        html.H4(
                            'Remote control',
                            style={'textAlign': 'left',
                                   'color': '#02467d',
                                   'font-weight': 700,
                                   'font-size':17
                            }
                        ),
                        html.Br(0.3),
                        html.Div(
                            className='control-item',
                            children = [
                                html.Span(
                                    'Long strategy : ',
                                    style = {
                                        'color': '#02467d',
                                        'font-weight': 700,
                                        'verticalAlign': 'middle',
                                        'marginRight': '20px',
                                        'marginLeft': '20px'
                                    }
                                ),
                                dcc.Dropdown(
                                    ['Yes', 'No'],
                                    'Yes',
                                    id='long-strategy-selection',
                                    style = {
                                        'hight': '200px',
                                        'position': 'relative',
                                        'display':'inline-block',
                                        'backgroundColor': '#e31096',
                                        'min-width': '80px',
                                        'borderRadius': '15px',
                                        'verticalAlign': 'middle'
                                    }
                                )
                            ]
                        ),
                        html.Div(
                            children = [
                                html.Br(),
                                html.Br(), 
                                html.Span(
                                    'Short strategy : ',
                                    style = {'color': '#02467d',
                                             "font-weight": 700,
                                             'verticalAlign': 'middle',
                                             'marginRight': '20px',
                                             'marginLeft': '20px'
                                            }
                                ),
                                dcc.Dropdown(
                                    ['Yes', 'No'],
                                    'Yes',
                                    id='short-strategy-selection',
                                    style = {
                                        'hight': '200px',
                                        'position': 'relative',
                                        'display':'inline-block',
                                        'backgroundColor': '#e31096',
                                        'min-width': '80px',
                                        'borderRadius': '15px',
                                        'verticalAlign': 'middle'
                                    }
                                )
                            ]
                        ),
                        html.Div(
                            children = [
                                html.Br(),
                                html.Br(), 
                                html.Span(
                                    'Both strategy : ',
                                    style = {
                                        'color': '#02467d',
                                        "font-weight": 700,
                                        'verticalAlign': 'middle',
                                        'marginRight': '20px',
                                        'marginLeft': '20px'
                                    }
                                ),
                                dcc.Dropdown(
                                    ['Yes', 'No'],
                                    'Yes',
                                    id='both-strategy-selection',
                                    style = {
                                        'hight': '200px',
                                        'position': 'relative',
                                        'display':'inline-block',
                                        'backgroundColor': '#e31096',
                                        'min-width': '80px',
                                        'borderRadius': '15px',
                                        'verticalAlign': 'middle'
                                    }
                                )
                            ]
                        ),
                        html.Div(
                            children = [
                                html.Br(),
                                html.Br(),
                                html.Br(),
                                html.Span(
                                    'Choose year',
                                    style={
                                        'textAlign': 'left',
                                        'color': '#02467d',
                                        'font-weight': 'bold',
                                        'font-size': 17
                                    }
                                ),
                                html.Div(
                                    style = {
                                        'margin': '10px'
                                    }
                                )
                            ]
                        ),
                        html.Div(
                            className = 'custom-slider',
                            children = [
                                dcc.Slider(
                                    id = 'year-slider',
                                    min = 0,
                                    max = number_of_years,
                                    value = 3,
                                    marks ={
                                        i: {
                                            'label': '{}'.format(int(YEARS[i])),
                                            'style': {'color': 'pink'}
                                        } if i != number_of_years else {
                                            'label': 'Total perf',
                                            'style': {'color': 'pink'}
                                        } for i in range(number_of_years+1)
                                    },
                                    step = 1
                                )
                            ]
                        ),
                        html.Div(
                            className = 'custom-bar',
                            children = [
                                html.Br(),
                                html.Br(),
                                html.Br(),
                                dcc.Graph(id = 'bar', figure={}
                                         )
                            ]
                        )
                    ]
                ),
                html.Div(
                    style = {'flex': '50%'},
                    children = [
                        dcc.Graph(
                            id = 'Historical-performance-graph',
                            figure = {}
                        )
                    ]
                )
            ]
)

#taking control of outputs
@app.callback(
    Output('Historical-performance-graph', 'figure'),
    Input('long-strategy-selection', 'value'),
    Input('short-strategy-selection', 'value'),
    Input('both-strategy-selection', 'value'),
    Input('year-slider', 'value')
)


def update_graph(long_strategy_visible, short_strategy_visible, both_strategies_visible, selected_year):
    traces = []
    year = int(YEARS[selected_year])
    if long_strategy_visible == 'Yes':
        df = Data[Data['year']==year].drop(
            ['avg_strategy_short','both_strategy'],
            axis = 1
        )
        number_of_weeks = len(df)
        WEEKS = [i for i in range(1,number_of_weeks+1)]
        long_strategy = df['avg_strategy_long']
        traces.append(
            go.Scatter(
                x=WEEKS,
                y=long_strategy,
                mode='lines',
                name='Long Strategy',
                line=dict(color='darkblue')
            )
        )
    if short_strategy_visible == 'Yes':
        df = Data[Data['year']==year].drop(
            ['avg_strategy_long','both_strategy'],
            axis = 1
        )
        number_of_weeks = len(df)
        WEEKS = [i for i in range(1,number_of_weeks+1)]
        short_strategy = df['avg_strategy_short']
        traces.append(
            go.Scatter(
                x=WEEKS,
                y=short_strategy,
                mode='lines',
                name='Short Strategy',
                line=dict(color='#4b97e3')
            )
        )
    if both_strategies_visible == 'Yes':
        df = Data[Data['year']==year].drop(
            ['avg_strategy_short','avg_strategy_long'],
            axis = 1
        )
        number_of_weeks = len(df)
        WEEKS = [i for i in range(1,number_of_weeks+1)]
        both_strategy = df['both_strategy']
        traces.append(
            go.Scatter(
                x=WEEKS,
                y=both_strategy,
                mode='lines',
                name='Both Strategy',
                line=dict(color='#fc08c7')
            )
        )
    
     
    # Create the graph layout
    layout = go.Layout(
        xaxis ={
            'title': 'Weekly basis',
            'title_font': {
                'color': "#0264a6",
                'family':'Bold',
                'size':13
            }
        },
        yaxis ={
            'title': 'cumulated Performance in %',
            'title_font': {
                'color': '#0264a6',
                'family':'Bold',
                'size':13
            }
        }
    )
    
    # Create the figure and update its layout
    fig_grph = go.Figure(data=traces, layout=layout)
    fig_grph.update_layout(
        plot_bgcolor='white',
        xaxis=dict(showgrid=False, showline=True,linecolor='gray'),
        yaxis=dict(showgrid=True,showline=True, linecolor='gray'),
        title={
        'text': f'Historical performance of SP in {year}',
        'x': 0.5, 
        'xanchor': 'center'
        }, 
        title_font = dict(color='#0264a6', size = 15, family='Bold'), 
        margin=dict(t=50)
    )
    fig_grph.update_xaxes(dtick=5)
    
    return fig_grph



@app.callback(
    Output('bar', 'figure'),
    Input('long-strategy-selection', 'value'),
    Input('short-strategy-selection', 'value'),
    Input('both-strategy-selection', 'value'),
    Input('year-slider', 'value')
)

def update_bar_graph(long_strategy_visible, short_strategy_visible, both_strategies_visible, selected_year):
    year = int(YEARS[selected_year])
    selected_strategies = []
    averages = []
    variance = []
    if long_strategy_visible == 'Yes':
        df = Data[Data['year']==year].drop(
            ['avg_strategy_short','both_strategy'],
            axis = 1
        )
        selected_strategies.append('Long strategy')
        averages.append(df['avg_strategy_long'].mean())
        variance.append(df['avg_strategy_long'].var())
    
    if short_strategy_visible == 'Yes':
        df = Data[Data['year']==year].drop(
            ['avg_strategy_long','both_strategy'],
            axis = 1
        )
        selected_strategies.append('Short strategy')
        averages.append(df['avg_strategy_short'].mean())
        variance.append(df['avg_strategy_short'].var())
        
    if both_strategies_visible == 'Yes':
        df = Data[Data['year']==year].drop(
            ['avg_strategy_short','avg_strategy_long'],
            axis = 1
        )
        selected_strategies.append('Both strategy')
        averages.append(df['both_strategy'].mean())
        variance.append(df['both_strategy'].var())
        
    traces = [
        go.Bar(
            x=selected_strategies,
            y=averages,
            name=f'average performance in {year}',
            marker_color='#9971a3'
        ),
        go.Bar(
            x=selected_strategies,
            y=variance,
            name='variance',
            marker_color='#f4f0f5'
        )
    ]
    
    
    layout = go.Layout(
        xaxis={
            'title': 'Strategy',
            'title_font': {
                    'color': 'black',
                    'family':'David',
                    'size':13
                
            }
        },
        yaxis={
            'title': 'Value in %',
            'title_font': {
                    'color': 'black',
                    'family':'David',
                    'size':13
            }
        }
    )
        
    fig_bar = go.Figure(data=traces, layout=layout)
    fig_bar.update_layout(plot_bgcolor='white',
                    xaxis=dict(showgrid=False, showline=True,linecolor='gray' ),
                    yaxis=dict(showgrid=True,showline=True, linecolor='gray'),
        title={
        'text': f'Average and Variance of performance in {year}',
        'x': 0.35, 
        'xanchor': 'center'
        },
        title_font = dict(color='black', size = 13, family='David'), 
        margin=dict(t=50)
    )
    
    
    return fig_bar



if __name__ == '__main__':
    app.run_server(port='8072')

Dash is running on http://127.0.0.1:8072/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8072
Press CTRL+C to quit
127.0.0.1 - - [01/Jun/2023 13:47:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2023 13:47:46] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Jun/2023 13:47:46] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Jun/2023 13:47:46] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Jun/2023 13:47:46] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Jun/2023 13:47:46] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2023 13:47:46] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2023 13:47:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2023 13:47:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2023 13:47:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2023 13:47:59] "POST /_dash-upd

127.0.0.1 - - [01/Jun/2023 13:49:01] "POST /_dash-update-component HTTP/1.1[2023-06-01 13:49:01,323] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/axel/anaconda3/lib/python3.10/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/axel/anaconda3/lib/python3.10/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/axel/anaconda3/lib/python3.10/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/axel/anaconda3/lib/python3.10/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/Users/axel/anaconda3/lib/python3.10/site-packages/dash/dash.py", line 1283, in dispatch
    ctx.run(
  File "/Users/axel/anaconda3/lib/python3.10/site-packages/dash/_callback.py", li